# Développement d'une API en Python en utilisant le Framework HUG

## Instalation de HUG

In [1]:
# %pip install hug -U

# Création de l'API

In [2]:
import hug, random, requests, json

api = hug.get(on_invalid=hug.redirect.not_found)
url_station = 'https://www.juleshaag.fr/devIA/devAPI/station_information.json'

@hug.get('/hello')
def hello():
    return 'Hello, world!'

@hug.get('/var')
def var():
    return random.randint(0, 100)

@api.urls('/var2', examples='nb=1')
def generate_random_values(nb: hug.types.number):
    values = [random.randint(-1000, 1000) for _ in range(nb)]
    data = {"values": values}
    return data

@api.urls('/calc/add', examples='number_1=1&number_2=2')
def add(number_1: hug.types.number, number_2: hug.types.number):
    return number_1 + number_2

@api.urls('/calc/prod', examples='number_1=1&number_2=2')
def prod(number_1: hug.types.number, number_2: hug.types.number):
    return number_1 * number_2

@api.urls('/img', examples='num=1')
def img(num: hug.types.number):
    return hug.redirect.to(f'https://www.juleshaag.fr/devIA/devAPI/{num}.png')

@api.urls('/station_velo', examples='id=1')
def station_velo(id: hug.types.number):
    data=requests.get(url_station).json()
    for content in data['data']['stations']:
            if id == int(content['station_id']):
                return content
    return {"error": "Station vélo non trouvée"}

@api.urls('/station_velo2', examples='id=1&addr=<random_content>&cap=<random_content>')
def station_velo(id: hug.types.number, addr: hug.types.text=None, cap: hug.types.text=None):
    data=requests.get(url_station).json()
    for content in data['data']['stations']:
        if id == int(content['station_id']):
            resultat = content
        else:
            resultat = {"error": "Station vélo non trouvée"}

    if addr and cap:
        resultat = {"address": content['address'], "capacity": content['capacity']}
    elif addr :
        resultat =  {"address": content['address']}
    elif cap :
        resultat = {"capacity": content['capacity']}
    return resultat

In [3]:
@api.urls('/stations_velo/toutes/cap', output=hug.output_format.json)
def get_total_capacity(capacity_filter: hug.types.text = ''):
    data=requests.get(url_station).json()
    
    stations = data['data']['stations']
    capacity_list = [station['capacity'] for station in stations if not capacity_filter]
    total_capacity = sum(capacity_list)

    capacity_dict = {f"station_id {station['station_id']}": station['capacity'] for station in stations if not capacity_filter}
    capacity_dict['total_capacity'] = total_capacity

    return capacity_dict

In [5]:
@hug.get("/stations_velo5/{id}/cap",output=hug.output_format.text)

def root(id: hug.types.number, cap : hug.types.text = ''):

    get = requests.get(f'https://www.juleshaag.fr/devIA/devAPI/station_information.json',stream=True)
    decoded_data = get.content.decode('utf-8')
    json_data = json.loads(decoded_data)
    stations = json_data['data']['stations']
    for item in stations:
        if item['station_id']==f"{id}":

            if cap == '' :

                return int(item['capacity'])

In [6]:
# #http://localhost:8000/stations_velo4/id/addr"

@hug.get("/stations_velo5/{id}/addr",output=hug.output_format.text)

def root(id: hug.types.number, addr : hug.types.text = ''):

    get = requests.get(f'https://www.juleshaag.fr/devIA/devAPI/station_information.json',stream=True)

    decoded_data = get.content.decode('utf-8')

    json_data = json.loads(decoded_data)

    stations = json_data['data']['stations']

    for item in stations:

        if item['station_id']==f"{id}":

            if addr == '' :

                return item['address']

## Lancer l' API

In [7]:
if __name__ == '__main__':
    hug.API(__name__).http.serve()


/#######################################################################\
          `.----``..-------..``.----.
         :/:::::--:---------:--::::://.
        .+::::----##/-/oo+:-##----:::://
        `//::-------/oosoo-------::://.       ##    ##  ##    ##    #####
          .-:------./++o/o-.------::-`   ```  ##    ##  ##    ##  ##
             `----.-./+o+:..----.     `.:///. ########  ##    ## ##
   ```        `----.-::::::------  `.-:::://. ##    ##  ##    ## ##   ####
  ://::--.``` -:``...-----...` `:--::::::-.`  ##    ##  ##   ##   ##    ##
  :/:::::::::-:-     `````      .:::::-.`     ##    ##    ####     ######
   ``.--:::::::.                .:::.`
         ``..::.                .::         EMBRACE THE APIs OF THE FUTURE
             ::-                .:-
             -::`               ::-                   VERSION 2.6.1
             `::-              -::`
              -::-`           -::-
\########################################################################/

 Copyr

127.0.0.1 - - [27/Apr/2023 17:22:49] "GET /station_velo?id=30 HTTP/1.1" 200 138
127.0.0.1 - - [27/Apr/2023 17:22:55] "GET /station_velo/toutes/cap HTTP/1.1" 404 6997
127.0.0.1 - - [27/Apr/2023 17:23:21] "GET /station_velo/stations_velo5/30/cap HTTP/1.1" 404 6997
127.0.0.1 - - [27/Apr/2023 17:23:54] "GET /station_velo/stations_velo5/id=30/cap HTTP/1.1" 404 6997


KeyboardInterrupt: 